In [ ]:
import torch
import cv2
import random
import numpy as np
import os
import torch.nn as nn  # 'nn'이라는 별칭으로 torch.nn 모듈을 가져옴
import torch.nn.functional as F  # 필요한 경우 함수형 API 사용
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image  # OpenCV 대신 PIL로 이미지를 읽고 처리
import pandas as pd
from tqdm.auto import tqdm

# 평가지표
- 정확도
- 1~3등 스벅쿠폰

# 데이터셋 다운로드 및 압축 풀기

In [ ]:
!gdown 1Cax_RFJqRkUYlOMyjHc5NGZ_kfd2j2r1
!unzip -oqq pizza_steak_sushi.zip

Downloading...
From (original): https://drive.google.com/uc?id=1Cax_RFJqRkUYlOMyjHc5NGZ_kfd2j2r1
From (redirected): https://drive.google.com/uc?id=1Cax_RFJqRkUYlOMyjHc5NGZ_kfd2j2r1&confirm=t&uuid=ce3d3762-569b-483b-8441-5794f4147465
To: /content/pizza_steak_sushi.zip
100% 158M/158M [00:01<00:00, 132MB/s]


In [ ]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
DATA_PATH = "data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 음식 분류 데이터셋
- 0 : 피자
- 1 : 스테이크
- 2 : 스시


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.shape , test.shape

((1649, 2), (1350, 2))

In [ ]:
train.head()

# 데이터 폴더 앞에 있는 거
# 그럼 data 를 filename 앞에 놓는다.

,file_name,target
0,2104569.jpg,0
1,2038418.jpg,1
2,1919810.jpg,2
3,2557340.jpg,0
4,3621562.jpg,1


In [ ]:
train_path = (DATA_PATH + train['file_name']).to_numpy()
test_path = (DATA_PATH+test['file_name']).to_numpy()
target = (train['target']).to_numpy()

In [ ]:
len(train_path)

1649

# 1. 데이터셋 클래스 만들기


In [ ]:
import torchvision.transforms as transforms
from PIL import Image

class FoodDataset(Dataset):
    def __init__(self, image_paths, labels=None, resize=(224, 224), augment=False):
        self.image_paths = image_paths
        self.labels = labels
        self.resize = resize
        self.augment = augment

        # Transform 정의
        self.transform = transforms.Compose([
            transforms.Resize(self.resize),
            transforms.RandomHorizontalFlip(p=0.5) if self.augment else transforms.Lambda(lambda x: x),
            transforms.RandomRotation(15) if self.augment else transforms.Lambda(lambda x: x),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            img = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None  # 또는 대체 데이터 반환

        img = self.transform(img)

        if self.labels is not None:
            label = self.labels[idx]
            return {"x": img, "y": torch.tensor(label, dtype=torch.long)}
        return {"x": img}


In [ ]:
import torch
import torch.nn as nn

class ImprovedFoodClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(ImprovedFoodClassifier, self).__init__()

        # Convolutional layers with BatchNorm and Dropout
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # Conv1
            nn.BatchNorm2d(32),  # BatchNorm1
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Pool1
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # Conv2
            nn.BatchNorm2d(64),  # BatchNorm2
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Pool2
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # Conv3
            nn.BatchNorm2d(128),  # BatchNorm3
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Pool3
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),  # Conv4
            nn.BatchNorm2d(256),  # BatchNorm4
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Pool4
        )

        # Global Average Pooling (GAP) for dynamic input size handling
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Fully Connected layers
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),  # Fully Connected 1
            nn.LeakyReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(128, num_classes),  # Fully Connected 2 (Output)
        )

    def forward(self, x):
        x = self.conv_layers(x)  # Pass through convolutional layers
        x = self.global_avg_pool(x)  # Apply Global Average Pooling
        x = self.fc_layers(x)  # Pass through fully connected layers
        return x


# 2. 학습 loop 함수 만들기

In [ ]:
def train_loop(dl, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()  # 모델을 학습 모드로 설정
    for batch in dl:
        # 입력 데이터와 레이블을 디바이스로 이동
        x = batch["x"].to(device)
        y = batch["y"].to(device)

        # 모델 예측
        pred = model(x)

        # 손실 계산
        loss = loss_fn(pred, y)

        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 손실 누적
        epoch_loss += loss.item()

    # 평균 손실 반환
    epoch_loss /= len(dl)
    return epoch_loss

# 3. 테스트 loop 함수 만들기
- 데이터 예측 기능 및 검증데이터 손실값 반환하는 기능

In [ ]:
@torch.no_grad()
def test_loop(dl, model, loss_fn, device):
    epoch_loss = 0
    model.eval()  # 모델을 평가 모드로 설정
    pred_list = []

    for batch in dl:
        # 입력 데이터를 디바이스로 이동
        x = batch["x"].to(device)
        y = batch["y"].to(device) if "y" in batch else None

        # 모델 예측
        pred = model(x)

        # 손실 계산 (레이블이 제공된 경우)
        if y is not None:
            loss = loss_fn(pred, y)
            epoch_loss += loss.item()

        # 확률 값으로 변환 (Softmax)
        pred = torch.softmax(pred, dim=1)
        pred_list.append(pred.cpu().numpy())

    # 예측 결과와 평균 손실 반환
    pred = np.concatenate(pred_list)
    epoch_loss /= len(dl)
    return epoch_loss, pred

# 4. 학습하기


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import torch
import numpy as np

# 하이퍼파라미터
n_splits = 5
batch_size = 32
epochs = 100
loss_fn = torch.nn.CrossEntropyLoss()  # 다중 클래스 손실 함수
cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

reset_seeds(SEED)  # 시드 고정
score_list = []

for i, (tri, vai) in enumerate(cv.split(train_path)):
    # 모델 초기화
    model = ImprovedFoodClassifier(num_classes=3).to(device)  # 다중 클래스 모델
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # 데이터셋 및 데이터로더 생성
    train_dt = FoodDataset(train_path[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, shuffle=True, batch_size=batch_size)

    valid_dt = FoodDataset(train_path[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, shuffle=False, batch_size=batch_size)

    best_score = 0
    patience = 0

    for epoch in range(epochs):
        # 학습 루프
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)

        # 검증 루프
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)

        # 다중 클래스 예측 (argmax로 클래스 선택)
        pred_classes = pred.argmax(axis=1)

        # 정확도 계산
        score = accuracy_score(target[vai], pred_classes)

        print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Accuracy: {score:.4f}")

        # Early Stopping 체크
        patience += 1
        if score > best_score:
            patience = 0
            best_score = score
            torch.save(model.state_dict(), f"model_fold_{i}.pt")  # 최적 모델 저장

        if patience == 5:  # Early Stopping
            break

    print(f"Fold-{i}: Best Accuracy: {best_score:.4f}")
    score_list.append(best_score)

print(f"Cross-Validation Average Accuracy: {np.mean(score_list):.4f}")


Epoch 1, Train Loss: 0.9456, Valid Loss: 1.2987, Accuracy: 0.4030
Epoch 2, Train Loss: 0.8273, Valid Loss: 1.7076, Accuracy: 0.3788
Epoch 3, Train Loss: 0.7791, Valid Loss: 1.0169, Accuracy: 0.5424
Epoch 4, Train Loss: 0.7209, Valid Loss: 0.8062, Accuracy: 0.6242
Epoch 5, Train Loss: 0.6571, Valid Loss: 0.9737, Accuracy: 0.6515
Epoch 6, Train Loss: 0.6653, Valid Loss: 0.9197, Accuracy: 0.5788
Epoch 7, Train Loss: 0.6160, Valid Loss: 0.6890, Accuracy: 0.7182
Epoch 8, Train Loss: 0.6006, Valid Loss: 0.9654, Accuracy: 0.6848
Epoch 9, Train Loss: 0.5715, Valid Loss: 0.6154, Accuracy: 0.7455
Epoch 10, Train Loss: 0.5595, Valid Loss: 0.6711, Accuracy: 0.7182
Epoch 11, Train Loss: 0.5111, Valid Loss: 0.6985, Accuracy: 0.7121
Epoch 12, Train Loss: 0.5457, Valid Loss: 0.6675, Accuracy: 0.7030
Epoch 13, Train Loss: 0.5258, Valid Loss: 1.3477, Accuracy: 0.5091
Epoch 14, Train Loss: 0.4820, Valid Loss: 0.8606, Accuracy: 0.7000
Fold-0: Best Accuracy: 0.7455
Epoch 1, Train Loss: 0.9615, Valid Loss: 

# 5. 테스트 데이터 예측

In [ ]:
# 테스트 데이터셋 생성
test_dt = FoodDataset(test_path)  # 레이블 없이 생성
test_dl = DataLoader(test_dt, batch_size=32, shuffle=False)

In [ ]:
# 가장 높은 점수를 기록한 Fold-1 모델 로드
model = ImprovedFoodClassifier(num_classes=3).to(device)
model.load_state_dict(torch.load("model_fold_4.pt"))
model.eval()  # 평가 모드로 설정

# 테스트 데이터 예측
predictions = []
with torch.no_grad():
    for batch in test_dl:
        x = batch["x"].to(device)
        outputs = model(x)
        preds = torch.argmax(torch.softmax(outputs, dim=1), dim=1)  # 클래스 인덱스 예측
        predictions.extend(preds.cpu().numpy())  # numpy 배열로 변환하여 저장

# 예측 결과를 DataFrame으로 변환
submission = pd.DataFrame({
    "file_name": test["file_name"],  # 테스트 데이터 파일 이름
    "target": predictions            # 예측된 클래스
})

# CSV 파일 저장
submission.to_csv("test2.csv", index=False)
print("예측 결과가 submission.csv로 저장되었습니다!")

<ipython-input-17-3e80427f12dc>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_fold_4.pt"))


예측 결과가 submission.csv로 저장되었습니다!


# 6. 칸스 사이트의 컴피티션 페이지에 제출하여 점수 확인해보세요.

In [ ]:
# pd.DataFrame(pred,columns = ['target']).to_csv('홍길동.csv', inde = Fasle)